<a href="https://colab.research.google.com/github/annhienktuit/UIT-AI-Challenge2020/blob/master/Count_movement_and_create_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage3: Count Movement and Create Submission

![alt text](https://github.com/hcmcaic/ai-challenge-2020/blob/master/sample_data/videos/sample_01.png?raw=true)

## Connect Colab with Google drive


In [12]:

# Mount "My Drive" into /content/drive
from google.colab import drive

google_drive_dir = 'CountMovements'  # @param

drive.mount('/content/drive')

mount_point = '/content/drive/My Drive/{}'.format(google_drive_dir)

# Change the root directory to your mount_point
% cd '$mount_point'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/CountMovements


## Setup

Download all file from the link: https://drive.google.com/drive/folders/1_BtRxstqKGs-54nrVCXRwEEhVhvnTbFw?usp=sharing and put them in your google_drive_dir

Put the tracking result into your google_drive_dir/Tracking_result

![alt text](https://scontent.fsgn2-6.fna.fbcdn.net/v/t1.15752-9/117049098_387940832169877_532602403009635312_n.png?_nc_cat=100&_nc_sid=b96e70&_nc_ohc=GOOyzc1VGKsAX-1S3tz&_nc_ht=scontent.fsgn2-6.fna&oh=8d1a4b1f2d642cc27abca1964d7505d6&oe=5F5890EF)

```
1,1,133.8361358642578,164.6830596923828,51.21794128417969,50.30419921875,-1,-1,-1,-1
2,1,134.30209350585938,165.528076171875,50.88421630859375,50.6221923828125,-1,-1,-1,-1
3,1,134.07371520996094,167.19955444335938,50.71952819824219,50.36436462402344,-1,-1,-1,-1
4,1,132.49212646484375,168.33074951171875,51.94300842285156,51.41900634765625,-1,-1,-1,-1
5,1,131.67698669433594,169.62181091308594,51.90150451660156,51.30799865722656,-1,-1,-1,-1
6,1,130.94476318359375,170.0286865234375,52.04766845703125,52.654266357421875,-1,-1,-1,-1
7,1,130.13148498535156,170.85205078125,52.57478332519531,53.91021728515625,-1,-1,-1,-1
......
```



The tracking_result has the following format:


```
<frame_id>,<label>,<x_center>,<y_center>,<diagonal>,<confidence_score>, <obj_type>
...
```



## Count movements

https://github.com/Lijun-Yu/zero_virus

Tracking result for sample0.mp4 can be download at: https://drive.google.com/file/d/1MEkvPxBchpgUtR-8_QlCwSVHd_F426xf/view?usp=sharing

###Import libraries and configuration


In [9]:
import os,json
import numpy as np
from PIL import Image, ImageDraw
from collections import namedtuple
from enum import IntEnum, auto
from scipy.ndimage import gaussian_filter1d
from scipy.special import expit  # pylint: disable=no-name-in-module
from scipy.stats import linregress, norm

TrackItem = namedtuple('TrackItem', ['frame_id', 'obj_type', 'data'])

Event = namedtuple('Event', [
    'video_id', 'frame_id', 'movement_id', 'obj_type', 'confidence', 'track_id',
    'track'])

class ObjectType(IntEnum):
    '''
        Loại 1: xe 2 bánh như xe đạp, xe máy
        Loại 2: xe 4-7 chỗ như xe hơi, taxi, xe bán tải…
        Loại 3: xe trên 7 chỗ như xe buýt, xe khách
        Loại 4: xe tải, container, xe cứu hỏa
    '''
    Bike = 1
    Car = 2
    Taxi = 2
    Bus = 3
    Truck = 4
    Container = 4
    
    
    
  
#Config
TRACKING_RESULT_PATH = "Tracking_result/tracking_result_sample0.txt" #@param {type:"string"}
ROI_PATH = "cam_0.txt" #@param {type:"string"}
MOI_PATH = "cam_0.json" #@param {type:"string"}
IMG_HEIGHT = 450 #@param {type:"integer"}
IMG_WIDTH = 800 #@param {type:"integer"}
FPS = 15 #@param {type:"number"}


In [13]:
!pwd

/content/drive/My Drive/CountMovements


Load data from tracking result file

In [14]:
def getTrackItemByTrackId(track_id,path):
    track_items = []
    with open(path,'r') as f:
        data = f.read().splitlines()
    for d in data:
        v = d.split(",")
        if int(v[1])==track_id:
            class_id = 1 if float(v[6]) == -1.0 else int(float(v[6]))
            confi = 1 if float(v[5]) == -1.0 else float(v[5])
            track_item = TrackItem(int(v[0]), class_id, (float(v[2]), float(v[3]), float(v[4]), confi))
            track_items.append(track_item)
    return track_items

def get_region_mask(region, height, width):
    img = Image.new('L', (width, height), 0)
    region = region.flatten().tolist()
    ImageDraw.Draw(img).polygon(region, outline=0, fill=255)
    mask = np.array(img).astype(np.bool)
    return mask
trackitem = getTrackItemByTrackId(1,TRACKING_RESULT_PATH)
print(trackitem)

[TrackItem(frame_id=0, obj_type=1, data=(159.44510650634766, 189.8351593017578, 71.78990157695834, 1)), TrackItem(frame_id=1, obj_type=1, data=(159.74420166015625, 190.83917236328125, 71.77610905435205, 1)), TrackItem(frame_id=2, obj_type=1, data=(159.43347930908203, 192.3817367553711, 71.47754727628717, 1)), TrackItem(frame_id=3, obj_type=1, data=(158.46363067626953, 194.04025268554688, 73.08892075955659, 1)), TrackItem(frame_id=4, obj_type=1, data=(157.62773895263672, 195.27581024169922, 72.98134622831218, 1)), TrackItem(frame_id=5, obj_type=1, data=(156.96859741210938, 196.35581970214844, 74.03669061655427, 1)), TrackItem(frame_id=6, obj_type=1, data=(156.41887664794922, 197.80715942382812, 75.3021870162082, 1)), TrackItem(frame_id=7, obj_type=1, data=(155.98198699951172, 199.03945922851562, 75.59970580191057, 1)), TrackItem(frame_id=8, obj_type=1, data=(155.40133666992188, 200.71076202392578, 76.48051656605095, 1)), TrackItem(frame_id=9, obj_type=1, data=(154.9968605041504, 202.606

In [15]:
def get_track(track_items, roi_path,min_length = 0.3,stride=1, gaussian_std = 0.3,speed_window=1,min_speed=10):
    img_height= IMG_HEIGHT
    img_width= IMG_WIDTH
    fps= FPS
    min_length = max(3, min_length * fps)
    speed_window = int(speed_window * fps // 2) * 2
    init_frame_id = track_items[0].frame_id
    length = track_items[-1].frame_id - init_frame_id + 1

    if length < min_length:
        return None
    if len(track_items) == length:
        interpolated_track = np.stack([t.data for t in track_items])
    else:
        interpolated_track = np.empty((length, len(track_items[0].data)))
        interpolated_track[:, 0] = -1
        for t in track_items:
            interpolated_track[t.frame_id - init_frame_id] = t.data
        for frame_i, state in enumerate(interpolated_track):
            if state[0] >= 0:
                continue
            for left in range(frame_i - 1, -1, -1):
                if interpolated_track[left, 0] >= 0:
                    left_state = interpolated_track[left]
                    break
            for right in range(frame_i + 1, interpolated_track.shape[0]):
                if interpolated_track[right, 0] >= 0:
                    right_state = interpolated_track[right]
                    break
            movement = right_state - left_state
            ratio = (frame_i - left) / (right - left)
            interpolated_track[frame_i] = left_state + ratio * movement
    if gaussian_std is not None:
        gaussian_std = gaussian_std * fps
        track = gaussian_filter1d(
            interpolated_track, gaussian_std, axis=0, mode='nearest')
    else:
        track = interpolated_track
    track = np.hstack([track, np.arange(
        init_frame_id, init_frame_id + length)[:, None]])
    speed_window = min(speed_window, track.shape[0] - 1)
    speed_window_half = speed_window // 2
    speed_window = speed_window_half * 2

    speed = np.linalg.norm(
        track[speed_window:, :2] - track[:-speed_window, :2], axis=1)
    speed_mask = np.zeros((track.shape[0]), dtype=np.bool)
    speed_mask[speed_window_half:-speed_window_half] = \
        speed >= min_speed
    speed_mask[:speed_window_half] = speed_mask[speed_window_half]
    speed_mask[-speed_window_half:] = speed_mask[-speed_window_half - 1]
    track = track[speed_mask]
    track_int = track[:, :2].round().astype(int)

    region = np.loadtxt(roi_path, delimiter=',',dtype=np.int)
    region_mask = get_region_mask(region, img_height, img_width)

    iou_mask = region_mask[track_int[:, 1], track_int[:, 0]]
    track = track[iou_mask]
    if track.shape[0] < min_length:
        return None
    return track
track = get_track(trackitem,ROI_PATH)
print(track)

[[158.75742334 192.2300419   72.52983266   1.           0.        ]
 [158.50804548 192.98055177  72.8136845    1.           1.        ]
 [158.2115915  193.8604809   73.16264786   1.           2.        ]
 [157.87348805 194.86560075  73.57744978   1.           3.        ]
 [157.50328913 195.98746336  74.0557478    1.           4.        ]
 [157.11407151 197.21463654  74.59275083   1.           5.        ]
 [156.7213806  198.5339797   75.18248645   1.           6.        ]
 [156.34201138 199.93184075  75.8192079    1.           7.        ]
 [155.99279213 201.39501422  76.49882889   1.           8.        ]
 [155.68956971 202.91147387  77.21975387   1.           9.        ]
 [155.44644422 204.47086322  77.98303621   1.          10.        ]
 [155.27543296 206.06488109  78.79202499   1.          11.        ]
 [155.18635723 207.68739005  79.65135433   1.          12.        ]
 [155.18694423 209.33426089  80.56598395   1.          13.        ]
 [155.28308672 211.0029813   81.54035694   1.   

Classify tracking obj into movement 

In [16]:
#Get movement_score
def get_movement_heatmaps(movements, height, width):
    distance_heatmaps = np.empty((len(movements), height, width))
    proportion_heatmaps = np.empty((len(movements), height, width))
    xs, ys = np.meshgrid(np.arange(width), np.arange(height))
    points = np.stack([xs.flatten(), ys.flatten()], axis=1)
    for label, movement_vertices in movements.items():
        vectors = movement_vertices[1:] - movement_vertices[:-1]
        lengths = np.linalg.norm(vectors, axis=-1) + 1e-4
        rel_lengths = lengths / lengths.sum()
        vertex_proportions = np.cumsum(rel_lengths)
        vertex_proportions = np.concatenate([[0], vertex_proportions[:-1]])
        offsets = ((points[:, None] - movement_vertices[None, :-1])
                   * vectors[None]).sum(axis=2)
        fractions = np.clip(offsets / (lengths ** 2), 0, 1)
        targets = movement_vertices[:-1] + fractions[:, :, None] * vectors
        distances = np.linalg.norm(points[:, None] - targets, axis=2)
        nearest_segment_ids = distances.argmin(axis=1)
        nearest_segment_fractions = fractions[
            np.arange(fractions.shape[0]), nearest_segment_ids]
        distance_heatmap = distances.min(axis=1)
        proportion_heatmap = vertex_proportions[nearest_segment_ids] + \
            rel_lengths[nearest_segment_ids] * nearest_segment_fractions
        distance_heatmaps[label - 1, ys, xs] = distance_heatmap.reshape(
            height, width)
        proportion_heatmaps[label - 1, ys, xs] = proportion_heatmap.reshape(
            height, width)
    return distance_heatmaps, proportion_heatmaps

def get_movement_scores(track, obj_type, moi_path, proportion_thres_to_delta=0.5, distance_base_size=4, distance_scale=5, start_period=0.3, start_thres=0.5,proportion_scale=0.8,distance_slope_scale=2,merge_detection_score=False,final=True):
    img_height= IMG_HEIGHT
    img_width= IMG_WIDTH
    fps=FPS
    with open(os.path.join(moi_path)) as f:
        data = json.load(f)
    movements = {int(shape['label']): np.array(shape['points']) for shape in data['shapes']}
    assert len(movements) == max(movements.keys())

    positions = track[:, :2].round().astype(int)
    diagonals = track[:, 2]
    detection_scores = track[:, 3]
    frame_ids = track[:, -1]

    distance_heatmaps, proportion_heatmaps =  get_movement_heatmaps(movements, img_height, img_width)

    distances = distance_heatmaps[:, positions[:, 1], positions[:, 0]]
    proportions = proportion_heatmaps[:, positions[:, 1], positions[:, 0]]
    distances = distances / diagonals[None]
    mean_distances = distances.mean(axis=1)
    x = np.linspace(0, 1, proportions.shape[1])
    distance_slopes = np.empty((len(movements)))
    proportion_slopes = np.empty((len(movements)))
    for movement_i in range(len(movements)):
        distance_slopes[movement_i] = linregress(
            x, distances[movement_i])[0]
        proportion_slopes[movement_i] = linregress(
            x, proportions[movement_i])[0]
    proportion_delta = proportions.max(axis=1) - proportions.min(axis=1)
    proportion_slopes = np.where(
        proportion_slopes >= proportion_thres_to_delta,
        proportion_delta, proportion_slopes)
    if obj_type == ObjectType.Truck:
        distance_base_scale = min(
            1, distance_base_size / mean_distances.shape[0])
        distance_base = np.sort(mean_distances)[
            :distance_base_size].sum() * distance_base_scale
        score_1 = 1 - (mean_distances / distance_base) ** 2
    else:
        score_1 = expit(4 - mean_distances * distance_scale)

    proportion_factor = 1 / proportion_scale
    score_2 = proportion_factor * np.minimum(
        proportion_slopes, 1 / (proportion_slopes + 1e-8))
    start_period=start_period*fps
    if frame_ids[0] <= start_period and \
            score_2.max() <= start_thres:
        score_2 *= start_proportion_factor
    score_3 = norm.pdf(distance_slopes * distance_slope_scale) / 0.4
    scores = np.stack([score_1, score_2, score_3], axis=1)
    if final:
        scores = np.clip(scores, 0, 1).prod(axis=1)
        if merge_detection_score:
            scores = scores * detection_scores.mean()
    return scores
movement_scores = get_movement_scores(track, 1,MOI_PATH)
print(movement_scores)

[1.70646182e-07 9.67697287e-01]


Run movements for one track_id

In [17]:
def get_obj_type(track_items, track):
    active_frame_ids = set(track[:, -1].tolist())
    obj_types = [t.obj_type for t in track_items
                 if t.frame_id in active_frame_ids]
    type_counts = np.bincount(obj_types)
    class_id = np.argmax(type_counts)

    #Bike:0, Car:1, Truck:2
    if class_id == 1:
        obj_type = ObjectType.Car
    elif class_id == 0:
        obj_type = ObjectType.Bike
    elif class_id == 2:
        obj_type = ObjectType.Truck

    return obj_type

def get_event(video_id, track_id, tracking_result_path, roi_path, moi_path, stride=1,min_score=0.25,return_all_events=False):
    '''
        input:
        - (int)video_id -- the id of video tracking
        - (int)track_id -- the label of tracking to get movement result
        - (string)tracking_result_path -- the tracking result file path
        output:
          A movements result
          Event = namedtuple('Event', ['video_id', 'frame_id', 'movement_id', 'obj_type', 'confidence', 'track_id', 'track'])
    '''
    #Get track_id items
    track_items = getTrackItemByTrackId(track_id,tracking_result_path)

    if track_items is None:
        return None

    track = get_track(track_items,roi_path)

    if track is None:
        return None

    obj_type = get_obj_type(track_items, track)
    frame_id = (track_items[-1][0] + 1) * stride

    #Get movement_scores
    movement_scores = get_movement_scores(track, obj_type,moi_path)

    max_index = movement_scores.argmax()
    max_score = movement_scores[max_index]
    if max_score < min_score:
        if not return_all_events:
            return None
        movement_id = 0
    else:
        movement_id = max_index + 1

    event = Event(video_id, frame_id, movement_id, obj_type, max_score, track_id, track_items)
    return event

event = get_event(0,1,TRACKING_RESULT_PATH,ROI_PATH,MOI_PATH)
print(event)

Event(video_id=0, frame_id=93, movement_id=2, obj_type=<ObjectType.Car: 2>, confidence=0.9676972869600903, track_id=1, track=[TrackItem(frame_id=0, obj_type=1, data=(159.44510650634766, 189.8351593017578, 71.78990157695834, 1)), TrackItem(frame_id=1, obj_type=1, data=(159.74420166015625, 190.83917236328125, 71.77610905435205, 1)), TrackItem(frame_id=2, obj_type=1, data=(159.43347930908203, 192.3817367553711, 71.47754727628717, 1)), TrackItem(frame_id=3, obj_type=1, data=(158.46363067626953, 194.04025268554688, 73.08892075955659, 1)), TrackItem(frame_id=4, obj_type=1, data=(157.62773895263672, 195.27581024169922, 72.98134622831218, 1)), TrackItem(frame_id=5, obj_type=1, data=(156.96859741210938, 196.35581970214844, 74.03669061655427, 1)), TrackItem(frame_id=6, obj_type=1, data=(156.41887664794922, 197.80715942382812, 75.3021870162082, 1)), TrackItem(frame_id=7, obj_type=1, data=(155.98198699951172, 199.03945922851562, 75.59970580191057, 1)), TrackItem(frame_id=8, obj_type=1, data=(155.4

**Run movements for multi track_id**

In [18]:
def get_multi_event(video_id,st_id,en_id,tracking_result_path, roi_path, moi_path):
    events = []
    for track_id in range(st_id,en_id+1):
        event = get_event(video_id,track_id,tracking_result_path, roi_path, moi_path)
        if event is None:
            continue
        events.append(event)
    events.sort(key=lambda x: x.frame_id)
    return events

video_id = 0
trackid_st = 1
trackid_en = 315 
results = get_multi_event(video_id,trackid_st,trackid_en,TRACKING_RESULT_PATH,ROI_PATH,MOI_PATH)

for d in results:
    print(d)

Event(video_id=0, frame_id=93, movement_id=2, obj_type=<ObjectType.Car: 2>, confidence=0.9676972869600903, track_id=1, track=[TrackItem(frame_id=0, obj_type=1, data=(159.44510650634766, 189.8351593017578, 71.78990157695834, 1)), TrackItem(frame_id=1, obj_type=1, data=(159.74420166015625, 190.83917236328125, 71.77610905435205, 1)), TrackItem(frame_id=2, obj_type=1, data=(159.43347930908203, 192.3817367553711, 71.47754727628717, 1)), TrackItem(frame_id=3, obj_type=1, data=(158.46363067626953, 194.04025268554688, 73.08892075955659, 1)), TrackItem(frame_id=4, obj_type=1, data=(157.62773895263672, 195.27581024169922, 72.98134622831218, 1)), TrackItem(frame_id=5, obj_type=1, data=(156.96859741210938, 196.35581970214844, 74.03669061655427, 1)), TrackItem(frame_id=6, obj_type=1, data=(156.41887664794922, 197.80715942382812, 75.3021870162082, 1)), TrackItem(frame_id=7, obj_type=1, data=(155.98198699951172, 199.03945922851562, 75.59970580191057, 1)), TrackItem(frame_id=8, obj_type=1, data=(155.4

## Create the submission

Submission format is defined at: http://aichallenge.hochiminhcity.gov.vn/huong-dan-nhom-1

In [19]:
def create_submission(results,result_file):
    with open(result_file,'w') as f:
        for d in results:
            f.write("{} {} {} {}\n".format(d.video_id,d.frame_id,d.movement_id,d.obj_type.value))
    print("the result is saved at " + result_file )
create_submission(results,'output.txt')


the result is saved at output.txt
